In [84]:
import numpy as np
import pandas as pd
import sentence_transformers.util, sentence_transformers.SentenceTransformer
import locale
locale.getpreferredencoding = lambda: "UTF-8"
import ast
from itertools import chain
 

In [2]:
library = pd.read_parquet('./final_data/library.parquet')

In [104]:
df =  pd.read_parquet('./data/filter_20k.parquet')
df.head()

,id,title_raw,abstract_raw,update_date,strip_cat,authors_parsed
182244,1412.3275,Limit cycles bifurcating from a degenerate center,We study the maximum number of limit cycles ...,2014-12-11,[DS],"[['Llibre', 'J.', ''], ['Pantazi', 'C.', '']]"
196425,0809.3510,Shrinking Point Bifurcations of Resonance Tong...,Resonance tongues are mode-locking regions o...,2015-05-13,[DS],"[['Simpson', 'D. J. W.', ''], ['Meiss', 'J. D...."
479424,2201.04222,Classification of Codimension-1 Singular Bifur...,The study of bifurcations of differential-al...,2022-01-13,[DS],"[['Ovsyannikov', 'Ivan', ''], ['Ruan', 'Haibo'..."
176385,1408.5812,Partial sums of excursions along random geodes...,"For a non-uniform lattice in SL(2,R), we con...",2014-10-09,"[GT, DS]","[['Gadre', 'Vaibhav', '']]"
291058,1707.03102,Uniform dimension results for a family of Mark...,In this paper we prove uniform Hausdorff and...,2017-10-03,[PR],"[['Sun', 'Xiaobin', ''], ['Xiao', 'Yimin', '']..."


In [3]:
library.head()

,index,id,title_raw,abstract_raw,update_date,strip_cat,authors_parsed,title_clean,abstract_clean,authors_clean,abstract_tokenized,abstract_reduced_tokens,abstract_rejoin,doc_string,doc_string_reduced,kaggle_index
0,182244,1412.3275,Limit cycles bifurcating from a degenerate center,We study the maximum number of limit cycles ...,2014-12-11,[DS],"[['Llibre', 'J.', ''], ['Pantazi', 'C.', '']]",limit cycles bifurcating from a degenerate center,we study the maximum number of limit cycles th...,"[['llibre', 'j', ''], ['pantazi', 'c', '']]","[we, study, the, maximum, number, of, limit, c...","[we, study, the, maximum, number, of, limit, c...",we study the maximum number of limit cycles th...,limit cycles bifurcating from a degenerate cen...,limit cycles bifurcating from a degenerate cen...,182244
1,196425,0809.3510,Shrinking Point Bifurcations of Resonance Tong...,Resonance tongues are mode-locking regions o...,2015-05-13,[DS],"[['Simpson', 'D. J. W.', ''], ['Meiss', 'J. D....",shrinking point bifurcations of resonance tong...,resonance tongues are mode locking regions of ...,"[['simpson', 'd j w', ''], ['meiss', 'j d', '']]","[resonance, tongues, are, mode, locking, regio...","[resonance, tongues, are, mode, locking, regio...",resonance tongues are mode locking regions of ...,shrinking point bifurcations of resonance tong...,shrinking point bifurcations of resonance tong...,196425
2,479424,2201.04222,Classification of Codimension-1 Singular Bifur...,The study of bifurcations of differential-al...,2022-01-13,[DS],"[['Ovsyannikov', 'Ivan', ''], ['Ruan', 'Haibo'...",classification of codimension singular bifurca...,the study of bifurcations of differential alge...,"[['ovsyannikov', 'ivan', ''], ['ruan', 'haibo'...","[the, study, of, bifurcations, of, differentia...","[the, study, of, bifurcations, of, differentia...",the study of bifurcations of differential alge...,classification of codimension singular bifurca...,classification of codimension singular bifurca...,479424
3,176385,1408.5812,Partial sums of excursions along random geodes...,"For a non-uniform lattice in SL(2,R), we con...",2014-10-09,"[GT, DS]","[['Gadre', 'Vaibhav', '']]",partial sums of excursions along random geodes...,for a non uniform lattice in slr we consider e...,"[['gadre', 'vaibhav', '']]","[for, a, non, uniform, lattice, in, slr, we, c...","[for, a, non, uniform, lattice, in, slr, we, c...",for a non uniform lattice in slr we consider e...,partial sums of excursions along random geodes...,partial sums of excursions along random geodes...,176385
4,291058,1707.03102,Uniform dimension results for a family of Mark...,In this paper we prove uniform Hausdorff and...,2017-10-03,[PR],"[['Sun', 'Xiaobin', ''], ['Xiao', 'Yimin', '']...",uniform dimension results for a family of mark...,in this paper we prove uniform hausdorff and p...,"[['sun', 'xiaobin', ''], ['xiao', 'yimin', '']...","[in, this, paper, we, prove, uniform, hausdorf...","[in, this, paper, we, prove, uniform, hausdorf...",in this paper we prove uniform hausdorff and p...,uniform dimension results for a family of mark...,uniform dimension results for a family of mark...,291058


In [70]:
#importing the list of topics
topics = pd.read_parquet('./final_data/library_topic_labels.parquet')
topics.head()

,fine_topic_labels
0,3 | bifurcation | bifurcations | hopf | hopf b...
1,3 | bifurcation | bifurcations | hopf | hopf b...
2,3 | bifurcation | bifurcations | hopf | hopf b...
3,-1
4,-1


In [74]:
#Obtaining all the topics as a list 
topics['Topic_list'] = topics['fine_topic_labels'].apply(lambda x: x.split(" | ")[1:])

In [76]:
# keeping only unique topics
def unique(input_list):
    final_list = []
    for word in input_list:
        if word not in final_list:
            final_list.append(word)
    return final_list
topics['Unique']=topics['Topic_list'].apply(lambda x:unique(x)) 

In [77]:
#function to check if one word is contained in another and it returns the compound word
def contained(word1,word2):
    if len(word1)>=len(word2):
        return word1
    else:
        if word1 in word2:
            return word2
        else:
            return word1
        
#function returning the unique sequences/topics
def unique_compound(input_list):
    final_list = []
    location =[]
    
    
    for i in range(len(input_list)):
        for j in range(i+1,len(input_list)):
            
            word = contained(input_list[i],input_list[j])
            if word!= input_list[i]:
                location.append(i)
                
    loc = list(set(location))
    for i in range(len(input_list)):
        if i not in location:
            final_list.append(input_list[i])
    
    return final_list
    
#final list of topics for each paper in our database    
topics['Unique']=topics['Unique'].apply(lambda x:str(unique_compound(x)))  
                
        

        

In [90]:
#topic recommender gives an array for topics of interest
def topic_recommender(topic_df,location_array):
    all_topics = []
    for i in range(5):
        if topic_df.Unique.iloc[location_array[i]]!='[]':
            all_topics.append(ast.literal_eval(topics.Unique.iloc[location_array[i]]))
#    print(all_topics)       
    if not all_topics:
        return ['Need new input']
#    all_topics = np.array(all_topics)
#    all_topics = list(set(np.reshape(all_topics,(1,-1))))
        
    return unique_compound(list(set(list(chain.from_iterable(all_topics)))))

In [87]:
topics = pd.read_parquet('./final_data/library_topic_labels_all.parquet')
topics.head()

,fine_topic_labels,Topic_list,Unique
0,3 | bifurcation | bifurcations | hopf | hopf b...,"[bifurcation, bifurcations, hopf, hopf bifurca...","['bifurcations', 'hopf bifurcation', 'paramete..."
1,3 | bifurcation | bifurcations | hopf | hopf b...,"[bifurcation, bifurcations, hopf, hopf bifurca...","['bifurcations', 'hopf bifurcation', 'paramete..."
2,3 | bifurcation | bifurcations | hopf | hopf b...,"[bifurcation, bifurcations, hopf, hopf bifurca...","['bifurcations', 'hopf bifurcation', 'paramete..."
3,-1,[],[]
4,-1,[],[]


In [91]:
topic_recommender(topics,[1,2,3,80,90])

['probability measures',
 'hopf bifurcation',
 'free',
 'periodic',
 'boolean',
 'supersymmetry',
 'convolutions',
 'systems',
 'monotone',
 'supergravity',
 'superspace',
 'susy',
 'supersymmetric quantum',
 'orbits',
 'parameter',
 'bifurcations',
 'limit cycles',
 'probability',
 'additive',
 'independence',
 'models']

In [78]:
topics.to_parquet('./final_data/library_topic_labels_all.parquet')

In [95]:
from transformers import pipeline
pipe = pipeline(model="facebook/bart-large-mnli")


Downloading:   0%|          | 0.00/6.38k [00:00<?, ?B/s]

In [96]:
#Using one-shot hot encoding on a sentence:
sentence = 'We give an analytic approach to the Saito vanishing theorem by going back to \
the original idea for the proof of the Kodaira-Nakano vanishing theorem. The key ingredient \
is the curvature formula for Hodge bundles and the Higgs field estimates for degenerations \
of Hodge structures.'

zeroshot_preds = pipe(sentence,candidate_labels=topic_recommender(topics,[1,2,3,80,90]))


In [97]:
print(zeroshot_preds)

{'sequence': 'We give an analytic approach to the Saito vanishing theorem by going back to the original idea for the proof of the Kodaira-Nakano vanishing theorem. The key ingredient is the curvature formula for Hodge bundles and the Higgs field estimates for degenerations of Hodge structures.', 'labels': ['susy', 'parameter', 'independence', 'models', 'additive', 'systems', 'hopf bifurcation', 'monotone', 'free', 'probability measures', 'bifurcations', 'convolutions', 'boolean', 'periodic', 'limit cycles', 'probability', 'superspace', 'orbits', 'supersymmetric quantum', 'supersymmetry', 'supergravity'], 'scores': [0.3271976411342621, 0.3011004328727722, 0.04517519101500511, 0.04366160184144974, 0.033248208463191986, 0.02975466102361679, 0.02675359509885311, 0.02343505620956421, 0.020194673910737038, 0.019007481634616852, 0.017980996519327164, 0.01676877960562706, 0.0164844561368227, 0.014841532334685326, 0.012834103778004646, 0.012642581015825272, 0.010980377905070782, 0.0101197669282

In [98]:
model = sentence_transformers.SentenceTransformer("all-MiniLM-L6-v2")

    #Encoding the title and summary of the input article
input_embedding = model.encode(sentence)
topic_embedding = model.encode(topic_recommender(topics,[1,2,3,80,90]))
topic_reco = sentence_transformers.util.semantic_search(query_embeddings=input_embedding,
                                         corpus_embeddings=topic_embedding,top_k=5)


In [100]:
print(topic_reco[0])

[{'corpus_id': 5, 'score': 0.1407250612974167}, {'corpus_id': 20, 'score': 0.11950124800205231}, {'corpus_id': 2, 'score': 0.1068754494190216}, {'corpus_id': 10, 'score': 0.07938030362129211}, {'corpus_id': 12, 'score': 0.07911689579486847}]


In [102]:
topic_reco_id = [recs['corpus_id'] for recs in topic_reco[0]]